# Préparer un voyage avec Kayak

## Récupération des données météo
Pour récupérer les données météo, on utilisera l'API d'Open Weather (https://openweathermap.org/api).

Or celle-ci fonctionne avec des coordonnées GPS, il faut donc récupérer pour chaque ville sa latitude et sa longitude.

On utilisera alors l'API https://nominatim.org/release-docs/develop/api/Search/ qui permet de récupérer des coordonnés en fonction d'une recherche par addresse.

In [18]:
import pandas as pd
import numpy as np
import requests

try:
    df = pd.read_csv("data/df.csv", sep=';', index_col=0)
except:
    df = pd.read_json("data/cities.json")
    df = df.rename(columns={0 : "City"})
    df["Latitude"] = np.nan
    df["Longitude"] = np.nan
    df["Prain"] = np.nan
    df["Vrain"] = np.nan
    df["Temp"] = np.nan


In [2]:
def get_coordinates(city):
    """Return tuple(lat, long)"""

    display(f"get_coordinates {city}")
    r = requests.get("https://nominatim.openstreetmap.org/search", params= {"city": city, "country": "FRANCE", "format": "jsonv2"})
    if (r.status_code == 200):
        json = r.json()[0]
        return (json["lat"], json["lon"])
    else:
        raise ValueError("API did not return 200")

In [19]:
for index, row in df.iterrows():
    if (pd.isna(row["Latitude"]) == False and pd.isna(row["Longitude"]) == False):
        continue

    try:
        coordinates = get_coordinates(row["City"])
        display(coordinates)
        df.loc[df['City'] == row["City"], ['Latitude']] = coordinates[0]
        df.loc[df['City'] == row["City"], ['Longitude']] = coordinates[1]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

,City,Latitude,Longitude,Prain,Vrain,Temp
0,Mont Saint Michel,48.635954,-1.511460,NaN,NaN,NaN
1,St Malo,48.649518,-2.026041,NaN,NaN,NaN
2,Bayeux,49.276462,-0.702474,NaN,NaN,NaN
3,Le Havre,49.493898,0.107973,NaN,NaN,NaN
4,Rouen,49.440459,1.093966,NaN,NaN,NaN
5,Paris,48.858890,2.320041,NaN,NaN,NaN
6,Amiens,49.894171,2.295695,NaN,NaN,NaN
7,Lille,50.636565,3.063528,NaN,NaN,NaN
8,Strasbourg,48.584614,7.750713,NaN,NaN,NaN
9,Chateau du Haut Koenigsbourg,48.249523,7.345492,NaN,NaN,NaN


Maintenant que l'on a recuéperer les coordonnées GPS de chaque ville, on peut récupérer les infomations météorologique de ces villes dans les 7 prochains jours.

In [8]:
weather_api_key = 'd18cb64895afdf1a4079d65e40103694'
weather_domain = 'https://api.openweathermap.org'

In [9]:
def get_weather(city, lat, long):
    """Return tuple(rain probabilty, rain volume in mm, day temperature)"""

    print(f"get_weather {city}")
    r = requests.get(f"{weather_domain}/data/2.5/onecall", params= {"lat": lat, "lon": long, "exclude": "current,minutely,hourly,alerts", "units": "metric", "appid": weather_api_key})
    display(r.content)
    if (r.status_code == 200):
        json = r.json()["daily"][6]
        return (json["pop"], json["rain"], json["temp"]["day"])
    else:
        raise ValueError("API did not return 200")

In [10]:
for index, row in df.iterrows():
    if (pd.isna(row["Prain"]) == False and pd.isna(row["Vrain"]) == False and pd.isna(row["Temp"]) == False):
        continue

    if (index > 0):
        continue

    try:
        weather = get_weather(row["City"], row["Latitude"], row["Longitude"])
        display(weather)
        df.loc[df['City'] == row["City"], ['Prain']] = weather[0]
        df.loc[df['City'] == row["City"], ['Vrain']] = weather[1]
        df.loc[df['City'] == row["City"], ['Temp']] = weather[2]
    except ValueError as e:
       display(f"{row['City']} : {e}")


df

get_weather Mont Saint Michel


b'{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}'

'Mont Saint Michel : API did not return 200'

,City,Latitude,Longitude,Prain,Vrain,Temp
0,Mont Saint Michel,48.6359541,-1.511459954959514,NaN,NaN,NaN
1,St Malo,48.649518,-2.0260409,NaN,NaN,NaN
2,Bayeux,49.2764624,-0.7024738,NaN,NaN,NaN
3,Le Havre,49.4938975,0.1079732,NaN,NaN,NaN
4,Rouen,49.4404591,1.0939658,NaN,NaN,NaN
5,Paris,48.8588897,2.3200410217200766,NaN,NaN,NaN
6,Amiens,49.8941708,2.2956951,NaN,NaN,NaN
7,Lille,50.6365654,3.0635282,NaN,NaN,NaN
8,Strasbourg,48.584614,7.7507127,NaN,NaN,NaN
9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923,NaN,NaN,NaN


In [14]:
# save to csv to not recall API when the kernel restart
df.to_csv("data/df.csv", sep=';')